In [1]:
FG_week = 'W46'

In [2]:
import os
import pandas as pd
import pyodbc
from sqlalchemy import create_engine, event
from urllib.parse import quote_plus
import urllib
import sqlalchemy

path = '//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/Upload'
os.chdir(path)

# Setting

In [3]:
poc_list_file = '//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/0_GS25_POC_Info.xlsx'
sku_list_file = '//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/1_GS25_SKU_Mapping.xlsx'

# Loading

In [4]:
file_list = os.listdir(path)
file_list_order = [file for file in file_list if file.endswith('xlsx')]
print(file_list_order)

['1108_1114.xlsx']


In [5]:
df = pd.read_excel(file_list_order[0], sheet_name=None)

In [6]:
df.keys()

dict_keys(['Index', '1', '2', '3', '4', '5', '6', '7'])

In [7]:
Index = df['Index'].iloc[1:, [1, 3]]
Index.columns = ['sheet_name', 'time']
Index['time'] = Index['time'].apply(lambda x: x.replace("Day Ending ", ""))
Index = Index.reset_index().iloc[:, 1:]
Index['YYYYMMDD'] = Index['time'].apply(lambda x: "20" + x[6:8] + x[0:2] + x[3:5])
Index = Index[['sheet_name', 'YYYYMMDD']]
Index

,sheet_name,YYYYMMDD
0,1,20211108
1,2,20211109
2,3,20211110
3,4,20211111
4,5,20211112
5,6,20211113
6,7,20211114


In [8]:
data = pd.DataFrame()
for idx in Index['sheet_name']:
    tmp = pd.DataFrame(data = df[idx].iloc[7:, :])
    tmp['YYYYMMDD'] = Index.loc[Index['sheet_name'] == idx, 'YYYYMMDD'].values[0]
    data = pd.concat([data, tmp], axis=0)
    tmp = []

In [9]:
data.columns = ['Geography', 'Product', 'Unit Sales', 'YYYYMMDD']
data = data[['Geography', 'Product', 'Unit Sales', 'YYYYMMDD']]
data

,Geography,Product,Unit Sales,YYYYMMDD
7,V1049-GS25강남동원점,8801858011017-오비)카스캔355ML,2,20211108
8,V1049-GS25강남동원점,8801858011048-오비)카스라이트캔355ML,1,20211108
9,V1049-GS25강남동원점,8801858011024-오비)카스캔500ML,4,20211108
10,V1049-GS25강남동원점,8801119740113-하이트진로)테라캔500ML,1,20211108
11,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,5,20211108
...,...,...,...,...
266111,V7V52-GS25검단본점,8801021223575-오비)덕덕구스IPA캔500ML,1,20211114
266112,V7V52-GS25검단본점,8801021220857-오비)구스아일랜드IPA캔473ML,1,20211114
266113,V7V52-GS25검단본점,8809556110933-제주)금성맥주캔500ML,1,20211114
266114,V7V52-GS25검단본점,8809556110728-문화)성산일출봉에일캔500ML,1,20211114


In [10]:
data['YYYYMMDD'].unique()

array(['20211108', '20211109', '20211110', '20211111', '20211112',
       '20211113', '20211114'], dtype=object)

# POC check

In [11]:
poc_list = pd.read_excel(poc_list_file)
poc_list.head()

,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
0,VP231-GS25성북마을점,VP231,GS25성북마을점,1.569946e+09,B,2000337566,Y,특수강북팀,나르샤,M성북동,신규,LDA,서울특별시,성북구,성북동,Seoul,서울 성북구 성북동1가 102,127.003878,37.589002
1,V3Q58-GS25성남코아루점,V3Q58,GS25성남코아루점,1.931515e+09,A,2000503410,Y,성남,미네르바,L모란동,신규,LDA,경기도,성남시 중원구,성남동,Sudo_KW,경기 성남시 중원구 성남동 2201,127.139912,37.435822
2,V5D44-GS25상대원혜성점,V5D44,GS25상대원혜성점,1.101503e+09,C,2000525005,Y,성남,미네르바,L성남동,신규,LDA,경기도,성남시 중원구,상대원1동,Sudo_KW,경기 성남시 중원구 상대원동 1547,127.163013,37.435857
3,V5D44-GS25상대원혜성점,V5D44,GS25상대원혜성점,1.101503e+09,C,2000525005,Y,성남,미네르바,L성남동,신규,LDA,경기도,성남시 중원구,상대원1동,Sudo_KW,경기 성남시 중원구 상대원동 1547,127.163013,37.435857
4,VX552-GS25상대원혜성점,VX552,GS25상대원혜성점,1.101503e+09,C,2000525005,N,성남,미네르바,L성남동,신규,LDA,경기도,성남시 중원구,상대원1동,Sudo_KW,경기 성남시 중원구 상대원동 1547,127.163013,37.435857


In [12]:
isin_filter = data['Geography'].isin(poc_list['full_name'])

if isin_filter.sum() == len(data):
    print("OK to Go")
else:
    missing_poc = data[~isin_filter]['Geography']
    print(data[~isin_filter]['Geography'])

71811     V7X88-GS25영통한아름점
71812     V7X88-GS25영통한아름점
71813     V7X88-GS25영통한아름점
71814     V7X88-GS25영통한아름점
71815     V7X88-GS25영통한아름점
                ...       
251317    V7Y28-GS25금왕사거리점
251318    V7Y28-GS25금왕사거리점
251319    V7Y28-GS25금왕사거리점
251320    V7Y28-GS25금왕사거리점
251321    V7Y28-GS25금왕사거리점
Name: Geography, Length: 1786, dtype: object


# SKU check

In [13]:
sku_list = pd.read_excel(sku_list_file)['Master_CD']
sku_list.head()

0     8801858042233-카스)이엑스병330ML*6
1       8801858042226-카스)이엑스병330ML
2    8801858044954-오비)카스캔740ML(3입)
3     8801858011116-오비)카스캔355MLX6팩
4         8801858133320-카스Q팩1.6LX6
Name: Master_CD, dtype: object

In [14]:
isin_filter = data['Product'].isin(sku_list)

if isin_filter.sum() == len(data):
    print("OK to Go")
else:
    missing_product = pd.DataFrame(data[~isin_filter]['Product'].unique(), columns=['Product'])
    print(missing_product)

                      Product
0    8809337550132-백석바이젠330ML
1   8809337550156-아미페일에일330ML
2    8809337550170-솔뫼IPA330ML
3  8809337550194-검은들스타우트330ML


# Validate if there is missing value in POCs or Product

In [15]:
missing_poc = pd.DataFrame(missing_poc).drop_duplicates()
missing_poc.columns = ['Geography']
missing_poc['poc_id'] = missing_poc['Geography'].apply(lambda x: x.split('-')[0])
missing_poc['poc_name'] = missing_poc['Geography'].apply(lambda x: x.split('-')[1])
missing_poc.head()

,Geography,poc_id,poc_name
71811,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점
27090,V7Y29-GS25합정당인점,V7Y29,GS25합정당인점
67573,V7Y43-GS25부천석원점,V7Y43,GS25부천석원점
199223,V7Y28-GS25금왕사거리점,V7Y28,GS25금왕사거리점
28307,V7Y84-GS25망원그린점,V7Y84,GS25망원그린점


In [16]:
method = 'GET'
key = 'KakaoAK b2fe4a3ff1e770cf35320c1e9b8d330e'
url = 'https://dapi.kakao.com/v2/local/search/keyword'
header = {'authorization' : key}

In [17]:
address_name = []
ID = []
place_name = []
road_address_name = []
X_CNTS = []
Y_CNTS = []

In [18]:
import tqdm
import requests

for poc in tqdm.tqdm(missing_poc['poc_name']):
    params = {'query': poc, 'category_group_code' : 'CS2'}
    response = requests.request(method=method, url=url, headers=header, params=params)
    token = response.json()
    if token['documents']:
        address_name.append(token['documents'][0].get('address_name'))
        ID.append(token['documents'][0].get('id'))
        place_name.append(token['documents'][0].get('place_name'))
        road_address_name.append(token['documents'][0].get('road_address_name'))
        X_CNTS.append(token['documents'][0].get('x'))
        Y_CNTS.append(token['documents'][0].get('y'))
        
    else:
        address_name.append('NA')
        ID.append('NA')
        place_name.append('NA')
        road_address_name.append('NA')
        X_CNTS.append('NA')
        Y_CNTS.append('NA')        

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 12.08it/s]


In [19]:
missing_poc = missing_poc.reset_index()
ID = pd.DataFrame(ID)
place_name = pd.DataFrame(place_name)
address_name = pd.DataFrame(address_name)
road_address_name = pd.DataFrame(road_address_name)
X_CNTS = pd.DataFrame(X_CNTS)
Y_CNTS = pd.DataFrame(Y_CNTS)

missing_poc = pd.concat([missing_poc, ID, place_name, address_name, road_address_name, X_CNTS, Y_CNTS], axis=1)

In [20]:
missing_poc.columns = ['index', 'full_name', 'poc_id', 'poc_name', 'Kakao_ID', 'place_name', 'address', 'road_address_name', 'X_CNTS', 'Y_CNTS']
missing_poc = missing_poc[['full_name', 'poc_id', 'poc_name', 'Kakao_ID', 'address', 'X_CNTS', 'Y_CNTS']]
missing_poc.head()

,full_name,poc_id,poc_name,Kakao_ID,address,X_CNTS,Y_CNTS
0,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1907359872,경기 수원시 영통구 매탄동 1277,127.043638006853,37.2566078507876
1,V7Y29-GS25합정당인점,V7Y29,GS25합정당인점,17056712,서울 마포구 합정동 358-2,126.917981574295,37.5466711672866
2,V7Y43-GS25부천석원점,V7Y43,GS25부천석원점,16168510,경기 부천시 심곡동 128-6,126.78760724884675,37.48416469356058
3,V7Y28-GS25금왕사거리점,V7Y28,GS25금왕사거리점,17056978,충북 음성군 금왕읍 무극리 144,127.596334651127,36.9939736424215
4,V7Y84-GS25망원그린점,V7Y84,GS25망원그린점,21763526,서울 마포구 망원동 475-52,126.903927594384,37.5603881687971


In [21]:
method = 'GET'
key = 'KakaoAK b2fe4a3ff1e770cf35320c1e9b8d330e'
url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode'
header = {'authorization' : key}

In [22]:
Sido = []
GuGun = []
UMD = []

for x, y in tqdm.tqdm(zip(missing_poc['X_CNTS'], missing_poc['Y_CNTS'])):
    params = {'x': x, 'y' : y}
    response = requests.request(method=method, url=url, headers=header, params=params)
    token = response.json()
    if x != 'NA':
        Sido.append(token['documents'][1]['region_1depth_name'])
        GuGun.append(token['documents'][1]['region_2depth_name'])
        UMD.append(token['documents'][1]['region_3depth_name'])
        
    else:
        Sido.append('NA')
        GuGun.append('NA')
        UMD.append('NA')
    
Sido = pd.DataFrame(Sido)
GuGun = pd.DataFrame(GuGun)
UMD = pd.DataFrame(UMD)

print(missing_poc.shape, Sido.shape, GuGun.shape, UMD.shape)

29it [00:02, 13.06it/s]

(29, 7) (29, 1) (29, 1) (29, 1)


In [23]:
missing_poc = pd.concat([missing_poc, Sido, GuGun, UMD], axis=1)

missing_poc.columns = ['full_name', 'poc_id', 'poc_name', 'Kakao_ID',
                       'address',
                       'X_CNTS', 'Y_CNTS',
                       'Sido', 'GuGun', 'UMD']

missing_poc = missing_poc[['full_name', 'poc_id', 'poc_name', 'Kakao_ID',
                           'Sido', 'GuGun', 'UMD', 'address',
                           'X_CNTS', 'Y_CNTS']]
missing_poc

,full_name,poc_id,poc_name,Kakao_ID,Sido,GuGun,UMD,address,X_CNTS,Y_CNTS
0,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1907359872,경기도,수원시 영통구,매탄3동,경기 수원시 영통구 매탄동 1277,127.043638006853,37.2566078507876
1,V7Y29-GS25합정당인점,V7Y29,GS25합정당인점,17056712,서울특별시,마포구,합정동,서울 마포구 합정동 358-2,126.917981574295,37.5466711672866
2,V7Y43-GS25부천석원점,V7Y43,GS25부천석원점,16168510,경기도,부천시,심곡동,경기 부천시 심곡동 128-6,126.78760724884675,37.48416469356058
3,V7Y28-GS25금왕사거리점,V7Y28,GS25금왕사거리점,17056978,충청북도,음성군,금왕읍,충북 음성군 금왕읍 무극리 144,127.596334651127,36.9939736424215
4,V7Y84-GS25망원그린점,V7Y84,GS25망원그린점,21763526,서울특별시,마포구,망원2동,서울 마포구 망원동 475-52,126.903927594384,37.5603881687971
5,V7Y62-GS25배곧한라점,V7Y62,GS25배곧한라점,365033553,경기도,시흥시,배곧2동,경기 시흥시 정왕동 2566,126.71446322350965,37.36419542198492
6,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1913434333,대구광역시,북구,구암동,대구 북구 구암동 688-6,128.555772734161,35.9274489999506
7,V7X97-GS25동명대로점,V7X97,GS25동명대로점,27499396,경상북도,칠곡군,동명면,경북 칠곡군 동명면 봉암리 193-1,128.553715539502,35.9749403501521
8,V7X55-GS25여의도현대점,V7X55,GS25여의도현대점,1844006595,서울특별시,영등포구,여의동,서울 영등포구 여의도동 2-2,126.92736271927402,37.52549298883295
9,V7Y88-GS25부평길주점,V7Y88,GS25부평길주점,1533293954,인천광역시,부평구,부개3동,인천 부평구 부개동 12-213,126.734328947598,37.5041583147752


In [24]:
tmp_poc = {'Sido' : ['서울특별시',
                     '강원도', '경기도', '인천광역시',
                     '경상북도', '대구광역시', '경상남도', '울산광역시', '부산광역시',
                     '충청북도', '충청남도', '세종특별자치시', '대전광역시', '전라북도', '전라남도', '광주광역시', '제주특별자치도'],
           'Region' : ['Seoul',
                       'Sudo_KW', 'Sudo_KW', 'Sudo_KW',
                       'East', 'East', 'East', 'East', 'East',
                       'West', 'West', 'West', 'West', 'West', 'West', 'West', 'West']}

tmp_region = pd.DataFrame(tmp_poc)

missing_poc = pd.merge(missing_poc, tmp_region, on='Sido', how='left')

In [25]:
missing_poc = missing_poc[['full_name', 'poc_id', 'poc_name', 'Kakao_ID', 
                           'Sido', 'GuGun', 'UMD', 'Region',
                           'address', 'X_CNTS', 'Y_CNTS']]

missing_poc.head()

,full_name,poc_id,poc_name,Kakao_ID,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
0,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1907359872,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638006853,37.2566078507876
1,V7Y29-GS25합정당인점,V7Y29,GS25합정당인점,17056712,서울특별시,마포구,합정동,Seoul,서울 마포구 합정동 358-2,126.917981574295,37.5466711672866
2,V7Y43-GS25부천석원점,V7Y43,GS25부천석원점,16168510,경기도,부천시,심곡동,Sudo_KW,경기 부천시 심곡동 128-6,126.78760724884675,37.48416469356058
3,V7Y28-GS25금왕사거리점,V7Y28,GS25금왕사거리점,17056978,충청북도,음성군,금왕읍,West,충북 음성군 금왕읍 무극리 144,127.596334651127,36.9939736424215
4,V7Y84-GS25망원그린점,V7Y84,GS25망원그린점,21763526,서울특별시,마포구,망원2동,Seoul,서울 마포구 망원동 475-52,126.903927594384,37.5603881687971


In [26]:
origin_poc = pd.read_excel(poc_list_file)
origin_poc.head()

,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
0,VP231-GS25성북마을점,VP231,GS25성북마을점,1.569946e+09,B,2000337566,Y,특수강북팀,나르샤,M성북동,신규,LDA,서울특별시,성북구,성북동,Seoul,서울 성북구 성북동1가 102,127.003878,37.589002
1,V3Q58-GS25성남코아루점,V3Q58,GS25성남코아루점,1.931515e+09,A,2000503410,Y,성남,미네르바,L모란동,신규,LDA,경기도,성남시 중원구,성남동,Sudo_KW,경기 성남시 중원구 성남동 2201,127.139912,37.435822
2,V5D44-GS25상대원혜성점,V5D44,GS25상대원혜성점,1.101503e+09,C,2000525005,Y,성남,미네르바,L성남동,신규,LDA,경기도,성남시 중원구,상대원1동,Sudo_KW,경기 성남시 중원구 상대원동 1547,127.163013,37.435857
3,V5D44-GS25상대원혜성점,V5D44,GS25상대원혜성점,1.101503e+09,C,2000525005,Y,성남,미네르바,L성남동,신규,LDA,경기도,성남시 중원구,상대원1동,Sudo_KW,경기 성남시 중원구 상대원동 1547,127.163013,37.435857
4,VX552-GS25상대원혜성점,VX552,GS25상대원혜성점,1.101503e+09,C,2000525005,N,성남,미네르바,L성남동,신규,LDA,경기도,성남시 중원구,상대원1동,Sudo_KW,경기 성남시 중원구 상대원동 1547,127.163013,37.435857


# 그래도 없는 POC들은 Master에서 발췌. 그래도 NA 나오면 수기 수정 필요

In [27]:
missing_poc = pd.merge(missing_poc, origin_poc[['poc_name', 'Grade', 'mpocs_cd', 'MNG_YN', 'Branch', 'Team', 'SR', 'New_Existed', 'Area']], on='poc_name', how='left')
missing_poc = missing_poc[['full_name', 'poc_id', 'poc_name', 'Kakao_ID', 'Grade', 'mpocs_cd',
                           'MNG_YN', 'Branch', 'Team', 'SR', 'New_Existed', 'Area', 'Sido',
                           'GuGun', 'UMD', 'Region', 'address', 'X_CNTS', 'Y_CNTS']]


In [28]:
missing_poc[['Grade', 'mpocs_cd', 'Branch', 'Team', 'SR', 'New_Existed']] = missing_poc[['Grade', 'mpocs_cd', 'Branch', 'Team', 'SR', 'New_Existed']].fillna('Not_allocated')
missing_poc['MNG_YN'] = missing_poc['MNG_YN'].fillna('N')

missing_poc = missing_poc.sort_values(by=['Y_CNTS'])
missing_poc.tail(20)

,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
12,V7X46-GS25풍세원룸점,V7X46,GS25풍세원룸점,916473095,D,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,충청남도,천안시 동남구,풍세면,West,충남 천안시 동남구 풍세면 용정리 853-95,127.09635215878,36.7326636947809
11,V7Y34-GS25수향연암로점,V7Y34,GS25수향연암로점,1856505501,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,충청남도,천안시 서북구,성환읍,West,충남 천안시 서북구 성환읍 수향리 434-27,127.133665998224,36.9488624200054
3,V7Y28-GS25금왕사거리점,V7Y28,GS25금왕사거리점,17056978,D,2000551923,Y,충주,푸른물결,M연수동B,신규,Residential,충청북도,음성군,금왕읍,West,충북 음성군 금왕읍 무극리 144,127.596334651127,36.9939736424215
0,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1907359872,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638006853,37.2566078507876
18,V7X89-GS25금곡센트럴점,V7X89,GS25금곡센트럴점,1990119616,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 권선구,금곡동,Sudo_KW,경기 수원시 권선구 금곡동 1072,126.945986295367,37.2746883824023
22,V7X93-GS25성복리버파크점,V7X93,GS25성복리버파크점,1913855999,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,용인시 수지구,상현2동,Sudo_KW,경기 용인시 수지구 성복동 1,127.08688473653655,37.31339720667861
19,V7X36-GS25의왕신안점,V7X36,GS25의왕신안점,2061522510,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,의왕시,고천동,Sudo_KW,경기 의왕시 왕곡동 592,126.98010524340263,37.34639439217768
5,V7Y62-GS25배곧한라점,V7Y62,GS25배곧한라점,365033553,A,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,경기도,시흥시,배곧2동,Sudo_KW,경기 시흥시 정왕동 2566,126.71446322350965,37.36419542198492
16,V7Y23-GS25금정중앙점,V7Y23,GS25금정중앙점,2036603616,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,군포시,산본1동,Sudo_KW,경기 군포시 산본동 236-15,126.939768814395,37.3698479980683
17,V7Y65-GS25송내성주로점,V7Y65,GS25송내성주로점,758598278,C,2000563469,Y,인천,퀸,L심곡동,신규,Office,경기도,부천시,대산동,Sudo_KW,경기 부천시 송내동 463-62,126.767099539265,37.4793921572676


In [29]:
origin_poc = pd.concat([origin_poc, missing_poc], axis=0)
origin_poc.tail(len(missing_poc))

,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
24,V7X72-GS25남천파크점,V7X72,GS25남천파크점,1994160573,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,부산광역시,수영구,남천1동,East,부산 수영구 남천동 24-21,129.1077250387438,35.14085544989688
10,V7X73-GS25함평월야점,V7X73,GS25함평월야점,1867910704,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라남도,함평군,월야면,West,전남 함평군 월야면 월야리 494-32,126.628737971708,35.1798196525737
20,V7Y56-GS25기장골드점,V7Y56,GS25기장골드점,17053199,D,8000068616,Y,부산,부산갈매기,M해운대우동,기존,Residential,부산광역시,기장군,기장읍,East,부산 기장군 기장읍 동부리 284-10,129.21693541675165,35.24566951098687
28,V7Y47-GS25우아시티점,V7Y47,GS25우아시티점,1779365518,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라북도,전주시 덕진구,우아1동,West,전북 전주시 덕진구 우아동3가 727-16,127.15335847481,35.8494748569445
6,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1913434333,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,대구광역시,북구,구암동,East,대구 북구 구암동 688-6,128.555772734161,35.9274489999506
7,V7X97-GS25동명대로점,V7X97,GS25동명대로점,27499396,D,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상북도,칠곡군,동명면,East,경북 칠곡군 동명면 봉암리 193-1,128.553715539502,35.9749403501521
21,V7Y26-GS25포항씨클리프점,V7Y26,GS25포항씨클리프점,1867957294,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 남구,동해면,East,경북 포항시 남구 동해면 입암리 549-1,129.472347770287,36.0110595305395
27,V7Y37-GS25포항화진점,V7Y37,GS25포항화진점,1790956525,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 북구,송라면,East,경북 포항시 북구 송라면 지경리 410-7,129.373260266161,36.2510623899598
26,V7Y49-GS25문경신기점,V7Y49,GS25문경신기점,1792554248,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,문경시,점촌4동,East,경북 문경시 신기동 1130,128.178538858801,36.6425367335491
12,V7X46-GS25풍세원룸점,V7X46,GS25풍세원룸점,916473095,D,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,충청남도,천안시 동남구,풍세면,West,충남 천안시 동남구 풍세면 용정리 853-95,127.09635215878,36.7326636947809


In [30]:
origin_poc.to_excel(poc_list_file, index=False, encoding='utf-8-sig')

# 여기 까지 하고 POC list 점검!!

# Trade area

In [31]:
origin_poc = pd.read_excel(poc_list_file)
origin_poc = origin_poc.sort_values(by='Area')
origin_poc.tail(15)

,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
25833,V7Y47-GS25우아시티점,V7Y47,GS25우아시티점,1.779366e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라북도,전주시 덕진구,우아1동,West,전북 전주시 덕진구 우아동3가 727-16,127.153358,35.849475
25834,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1.913434e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,대구광역시,북구,구암동,East,대구 북구 구암동 688-6,128.555773,35.927449
25836,V7Y26-GS25포항씨클리프점,V7Y26,GS25포항씨클리프점,1.867957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 남구,동해면,East,경북 포항시 남구 동해면 입암리 549-1,129.472348,36.011060
25837,V7Y37-GS25포항화진점,V7Y37,GS25포항화진점,1.790957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 북구,송라면,East,경북 포항시 북구 송라면 지경리 410-7,129.373260,36.251062
25838,V7Y49-GS25문경신기점,V7Y49,GS25문경신기점,1.792554e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,문경시,점촌4동,East,경북 문경시 신기동 1130,128.178539,36.642537
25840,V7Y34-GS25수향연암로점,V7Y34,GS25수향연암로점,1.856506e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,충청남도,천안시 서북구,성환읍,West,충남 천안시 서북구 성환읍 수향리 434-27,127.133666,36.948862
25842,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1.907360e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638,37.256608
25843,V7X89-GS25금곡센트럴점,V7X89,GS25금곡센트럴점,1.990120e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 권선구,금곡동,Sudo_KW,경기 수원시 권선구 금곡동 1072,126.945986,37.274688
25844,V7X93-GS25성복리버파크점,V7X93,GS25성복리버파크점,1.913856e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,용인시 수지구,상현2동,Sudo_KW,경기 용인시 수지구 성복동 1,127.086885,37.313397
25845,V7X36-GS25의왕신안점,V7X36,GS25의왕신안점,2.061523e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,의왕시,고천동,Sudo_KW,경기 의왕시 왕곡동 592,126.980105,37.346394


In [32]:
n = len(origin_poc.query("Area != Area"))
trade_poc = origin_poc.iloc[-n:, :]
trade_poc

,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
25830,V7X72-GS25남천파크점,V7X72,GS25남천파크점,1.994161e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,부산광역시,수영구,남천1동,East,부산 수영구 남천동 24-21,129.107725,35.140855
25831,V7X73-GS25함평월야점,V7X73,GS25함평월야점,1.867911e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라남도,함평군,월야면,West,전남 함평군 월야면 월야리 494-32,126.628738,35.179820
25833,V7Y47-GS25우아시티점,V7Y47,GS25우아시티점,1.779366e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라북도,전주시 덕진구,우아1동,West,전북 전주시 덕진구 우아동3가 727-16,127.153358,35.849475
25834,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1.913434e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,대구광역시,북구,구암동,East,대구 북구 구암동 688-6,128.555773,35.927449
25836,V7Y26-GS25포항씨클리프점,V7Y26,GS25포항씨클리프점,1.867957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 남구,동해면,East,경북 포항시 남구 동해면 입암리 549-1,129.472348,36.011060
25837,V7Y37-GS25포항화진점,V7Y37,GS25포항화진점,1.790957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 북구,송라면,East,경북 포항시 북구 송라면 지경리 410-7,129.373260,36.251062
25838,V7Y49-GS25문경신기점,V7Y49,GS25문경신기점,1.792554e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,문경시,점촌4동,East,경북 문경시 신기동 1130,128.178539,36.642537
25840,V7Y34-GS25수향연암로점,V7Y34,GS25수향연암로점,1.856506e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,충청남도,천안시 서북구,성환읍,West,충남 천안시 서북구 성환읍 수향리 434-27,127.133666,36.948862
25842,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1.907360e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638,37.256608
25843,V7X89-GS25금곡센트럴점,V7X89,GS25금곡센트럴점,1.990120e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 권선구,금곡동,Sudo_KW,경기 수원시 권선구 금곡동 1072,126.945986,37.274688


In [33]:
trade_poc['Index'] = trade_poc['Sido'] + "_" + trade_poc['GuGun']
trade_poc

C:\Users\15156\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS,Index
25830,V7X72-GS25남천파크점,V7X72,GS25남천파크점,1.994161e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,부산광역시,수영구,남천1동,East,부산 수영구 남천동 24-21,129.107725,35.140855,부산광역시_수영구
25831,V7X73-GS25함평월야점,V7X73,GS25함평월야점,1.867911e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라남도,함평군,월야면,West,전남 함평군 월야면 월야리 494-32,126.628738,35.179820,전라남도_함평군
25833,V7Y47-GS25우아시티점,V7Y47,GS25우아시티점,1.779366e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라북도,전주시 덕진구,우아1동,West,전북 전주시 덕진구 우아동3가 727-16,127.153358,35.849475,전라북도_전주시 덕진구
25834,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1.913434e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,대구광역시,북구,구암동,East,대구 북구 구암동 688-6,128.555773,35.927449,대구광역시_북구
25836,V7Y26-GS25포항씨클리프점,V7Y26,GS25포항씨클리프점,1.867957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 남구,동해면,East,경북 포항시 남구 동해면 입암리 549-1,129.472348,36.011060,경상북도_포항시 남구
25837,V7Y37-GS25포항화진점,V7Y37,GS25포항화진점,1.790957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,포항시 북구,송라면,East,경북 포항시 북구 송라면 지경리 410-7,129.373260,36.251062,경상북도_포항시 북구
25838,V7Y49-GS25문경신기점,V7Y49,GS25문경신기점,1.792554e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,문경시,점촌4동,East,경북 문경시 신기동 1130,128.178539,36.642537,경상북도_문경시
25840,V7Y34-GS25수향연암로점,V7Y34,GS25수향연암로점,1.856506e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,충청남도,천안시 서북구,성환읍,West,충남 천안시 서북구 성환읍 수향리 434-27,127.133666,36.948862,충청남도_천안시 서북구
25842,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1.907360e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638,37.256608,경기도_수원시 영통구
25843,V7X89-GS25금곡센트럴점,V7X89,GS25금곡센트럴점,1.990120e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경기도,수원시 권선구,금곡동,Sudo_KW,경기 수원시 권선구 금곡동 1072,126.945986,37.274688,경기도_수원시 권선구


In [34]:
master = pd.read_excel('//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/Trade_area_master.xlsx')
area = master['Index'].unique()

In [35]:
master

,Sido,GuGun,Index,Address,X_CNTS,Y_CNTS,Type
0,경기도,양주시,경기도_양주시,경기 양주시 덕정동 208 2층,127.065934,37.837234,Residential
1,경기도,양주시,경기도_양주시,경기 양주시 광적면 가납리 732-13,126.985997,37.822883,Residential
2,경기도,양주시,경기도_양주시,경기 양주시 옥정동 963-1,127.092755,37.822135,Residential
3,경기도,양주시,경기도_양주시,경기 양주시 옥정동 963-3,127.092144,37.821742,Residential
4,경기도,양주시,경기도_양주시,경기 양주시 옥정동 965-1,127.092829,37.821620,Residential
...,...,...,...,...,...,...,...
33485,제주특별자치도,서귀포시,제주특별자치도_서귀포시,제주도 서귀포시 칠십리로105번지,126.566306,33.241702,Residential
33486,제주특별자치도,서귀포시,제주특별자치도_서귀포시,제주도 서귀포시 칠십리로123번지,126.567575,33.243069,Residential
33487,제주특별자치도,서귀포시,제주특별자치도_서귀포시,제주도 서귀포시 칠십리로129번지,126.567865,33.243277,Residential
33488,제주특별자치도,서귀포시,제주특별자치도_서귀포시,제주도 서귀포시 칠십리로133번지,126.568226,33.243668,Residential


In [36]:
tmp_2 = origin_poc.iloc[:-n, :][['Sido', 'GuGun', 'address', 'X_CNTS', 'Y_CNTS', 'Area']]
tmp_2['Index'] = tmp_2['Sido'] + '_' + tmp_2['GuGun']
tmp_2 = tmp_2[['Sido', 'GuGun', 'Index', 'address', 'X_CNTS', 'Y_CNTS', 'Area']]
tmp_2

,Sido,GuGun,Index,address,X_CNTS,Y_CNTS,Area
0,서울특별시,성북구,서울특별시_성북구,서울 성북구 성북동1가 102,127.003878,37.589002,LDA
3924,경기도,고양시 일산동구,경기도_고양시 일산동구,경기 고양시 일산동구 장항동 854,126.773891,37.656418,LDA
3923,경기도,수원시 팔달구,경기도_수원시 팔달구,경기 수원시 팔달구 인계동 939-11,127.031332,37.276883,LDA
3922,경기도,수원시 팔달구,경기도_수원시 팔달구,경기 수원시 팔달구 인계동 956,127.028120,37.274869,LDA
3921,경기도,수원시 팔달구,경기도_수원시 팔달구,경기 수원시 팔달구 인계동 956,127.028120,37.274869,LDA
...,...,...,...,...,...,...,...
17413,경기도,평택시,경기도_평택시,경기 평택시 지산동 1238-1,127.076560,37.080270,Residential
17414,충청북도,단양군,충청북도_단양군,충북 단양군 매포읍 평동리 1136,128.298860,37.034119,Residential
17415,경기도,안산시 상록구,경기도_안산시 상록구,경기 안산시 상록구 사동 1384-1,126.846111,37.286108,Residential
17417,경기도,안산시 단원구,경기도_안산시 단원구,경기 안산시 단원구 대부북동 1848-116,126.572419,37.286023,Residential


In [37]:
tmp_2.rename(columns = {'address' : 'Address',
                        'Area' : 'Type'}, inplace=True)

master = pd.concat([master, tmp_2], axis=0)
master

,Sido,GuGun,Index,Address,X_CNTS,Y_CNTS,Type
0,경기도,양주시,경기도_양주시,경기 양주시 덕정동 208 2층,127.065934,37.837234,Residential
1,경기도,양주시,경기도_양주시,경기 양주시 광적면 가납리 732-13,126.985997,37.822883,Residential
2,경기도,양주시,경기도_양주시,경기 양주시 옥정동 963-1,127.092755,37.822135,Residential
3,경기도,양주시,경기도_양주시,경기 양주시 옥정동 963-3,127.092144,37.821742,Residential
4,경기도,양주시,경기도_양주시,경기 양주시 옥정동 965-1,127.092829,37.821620,Residential
...,...,...,...,...,...,...,...
17413,경기도,평택시,경기도_평택시,경기 평택시 지산동 1238-1,127.076560,37.080270,Residential
17414,충청북도,단양군,충청북도_단양군,충북 단양군 매포읍 평동리 1136,128.298860,37.034119,Residential
17415,경기도,안산시 상록구,경기도_안산시 상록구,경기 안산시 상록구 사동 1384-1,126.846111,37.286108,Residential
17417,경기도,안산시 단원구,경기도_안산시 단원구,경기 안산시 단원구 대부북동 1848-116,126.572419,37.286023,Residential


In [38]:
import numpy as np
data_2 = []
for area_name in tqdm.tqdm(area): #Area 별로 나눠서 search 횟수 줄임
    master_region = master.query("Index == @area_name")
    trade_poc_region = trade_poc.query("Index == @area_name")
    
    for x_cnts_region, y_cnts_region in zip(trade_poc_region['X_CNTS'], trade_poc_region['Y_CNTS']):
        a = [x_cnts_region, y_cnts_region]
        b = np.array(master_region['X_CNTS']).reshape(-1,1)
        c = np.array(master_region['Y_CNTS']).reshape(-1,1)
        d = np.append(b, c, axis=1)
        distance = (d-a)**2 # Euclidean 계산
        distance_2 = (distance[:, 0] + distance[:, 1])
        e = np.hstack([b,c,distance_2.reshape(-1,1)])
        minimum = np.min(distance_2)
        f = e[np.where(e[:, 2] == minimum)] # Index만 부름
        g = f[0, 0]
        h = f[0, 1]
        trade = master_region.query("(X_CNTS == @g) and (Y_CNTS == @h)")['Type']
        data_2.append(np.hstack([x_cnts_region, y_cnts_region, trade]))

100%|███████████████████████████████████████████████████████████████████████████████| 159/159 [00:00<00:00, 220.93it/s]


In [39]:
update_trade = pd.DataFrame(data_2)

update_trade = update_trade.iloc[:, :3]
update_trade.columns = ['X_CNTS', 'Y_CNTS', 'Area']
update_trade

,X_CNTS,Y_CNTS,Area
0,127.072799,37.621780,LDA
1,126.927363,37.525493,Office
2,126.980105,37.346394,Residential
3,126.939769,37.369848,Office
4,127.086885,37.313397,Residential
5,127.043638,37.256608,Office
6,126.945986,37.274688,Residential
7,127.133666,36.948862,LDA
8,129.373260,36.251062,Residential
9,129.472348,36.011060,Residential


In [40]:
tmp = []
tmp = pd.merge(trade_poc, update_trade, left_on=['X_CNTS', 'Y_CNTS'], right_on=['X_CNTS', 'Y_CNTS'], how='left')
tmp

,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,...,Area_x,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS,Index,Area_y
0,V7X72-GS25남천파크점,V7X72,GS25남천파크점,1.994161e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,부산광역시,수영구,남천1동,East,부산 수영구 남천동 24-21,129.107725,35.140855,부산광역시_수영구,Office
1,V7X73-GS25함평월야점,V7X73,GS25함평월야점,1.867911e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,전라남도,함평군,월야면,West,전남 함평군 월야면 월야리 494-32,126.628738,35.179820,전라남도_함평군,NaN
2,V7Y47-GS25우아시티점,V7Y47,GS25우아시티점,1.779366e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,전라북도,전주시 덕진구,우아1동,West,전북 전주시 덕진구 우아동3가 727-16,127.153358,35.849475,전라북도_전주시 덕진구,Residential
3,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1.913434e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,대구광역시,북구,구암동,East,대구 북구 구암동 688-6,128.555773,35.927449,대구광역시_북구,LDA
4,V7Y26-GS25포항씨클리프점,V7Y26,GS25포항씨클리프점,1.867957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,경상북도,포항시 남구,동해면,East,경북 포항시 남구 동해면 입암리 549-1,129.472348,36.011060,경상북도_포항시 남구,Residential
5,V7Y37-GS25포항화진점,V7Y37,GS25포항화진점,1.790957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,경상북도,포항시 북구,송라면,East,경북 포항시 북구 송라면 지경리 410-7,129.373260,36.251062,경상북도_포항시 북구,Residential
6,V7Y49-GS25문경신기점,V7Y49,GS25문경신기점,1.792554e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,경상북도,문경시,점촌4동,East,경북 문경시 신기동 1130,128.178539,36.642537,경상북도_문경시,NaN
7,V7Y34-GS25수향연암로점,V7Y34,GS25수향연암로점,1.856506e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,충청남도,천안시 서북구,성환읍,West,충남 천안시 서북구 성환읍 수향리 434-27,127.133666,36.948862,충청남도_천안시 서북구,LDA
8,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1.907360e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638,37.256608,경기도_수원시 영통구,Office
9,V7X89-GS25금곡센트럴점,V7X89,GS25금곡센트럴점,1.990120e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,...,NaN,경기도,수원시 권선구,금곡동,Sudo_KW,경기 수원시 권선구 금곡동 1072,126.945986,37.274688,경기도_수원시 권선구,Residential


In [41]:
tmp = tmp[['full_name', 'poc_id', 'poc_name', 'Kakao_ID', 'Grade', 'mpocs_cd',
       'MNG_YN', 'Branch', 'Team', 'SR', 'New_Existed', 'Area_y', 'Sido',
       'GuGun', 'UMD', 'Region', 'address', 'X_CNTS', 'Y_CNTS']]

In [42]:
tmp.rename(columns = {'Area_y' : 'Area'}, inplace=True)
tmp

C:\Users\15156\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
0,V7X72-GS25남천파크점,V7X72,GS25남천파크점,1.994161e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,부산광역시,수영구,남천1동,East,부산 수영구 남천동 24-21,129.107725,35.140855
1,V7X73-GS25함평월야점,V7X73,GS25함평월야점,1.867911e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,전라남도,함평군,월야면,West,전남 함평군 월야면 월야리 494-32,126.628738,35.179820
2,V7Y47-GS25우아시티점,V7Y47,GS25우아시티점,1.779366e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,전라북도,전주시 덕진구,우아1동,West,전북 전주시 덕진구 우아동3가 727-16,127.153358,35.849475
3,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1.913434e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,대구광역시,북구,구암동,East,대구 북구 구암동 688-6,128.555773,35.927449
4,V7Y26-GS25포항씨클리프점,V7Y26,GS25포항씨클리프점,1.867957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상북도,포항시 남구,동해면,East,경북 포항시 남구 동해면 입암리 549-1,129.472348,36.011060
5,V7Y37-GS25포항화진점,V7Y37,GS25포항화진점,1.790957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상북도,포항시 북구,송라면,East,경북 포항시 북구 송라면 지경리 410-7,129.373260,36.251062
6,V7Y49-GS25문경신기점,V7Y49,GS25문경신기점,1.792554e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,NaN,경상북도,문경시,점촌4동,East,경북 문경시 신기동 1130,128.178539,36.642537
7,V7Y34-GS25수향연암로점,V7Y34,GS25수향연암로점,1.856506e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,충청남도,천안시 서북구,성환읍,West,충남 천안시 서북구 성환읍 수향리 434-27,127.133666,36.948862
8,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1.907360e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638,37.256608
9,V7X89-GS25금곡센트럴점,V7X89,GS25금곡센트럴점,1.990120e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경기도,수원시 권선구,금곡동,Sudo_KW,경기 수원시 권선구 금곡동 1072,126.945986,37.274688


In [43]:
tmp['Area'][tmp['Area'].isna()] = 'Residential'
tmp

C:\Users\15156\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\15156\Anaconda3\lib\site-packages\pandas\core\series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


,full_name,poc_id,poc_name,Kakao_ID,Grade,mpocs_cd,MNG_YN,Branch,Team,SR,New_Existed,Area,Sido,GuGun,UMD,Region,address,X_CNTS,Y_CNTS
0,V7X72-GS25남천파크점,V7X72,GS25남천파크점,1.994161e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,부산광역시,수영구,남천1동,East,부산 수영구 남천동 24-21,129.107725,35.140855
1,V7X73-GS25함평월야점,V7X73,GS25함평월야점,1.867911e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,전라남도,함평군,월야면,West,전남 함평군 월야면 월야리 494-32,126.628738,35.179820
2,V7Y47-GS25우아시티점,V7Y47,GS25우아시티점,1.779366e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,전라북도,전주시 덕진구,우아1동,West,전북 전주시 덕진구 우아동3가 727-16,127.153358,35.849475
3,V7X68-GS25대구구암센터점,V7X68,GS25대구구암센터점,1.913434e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,대구광역시,북구,구암동,East,대구 북구 구암동 688-6,128.555773,35.927449
4,V7Y26-GS25포항씨클리프점,V7Y26,GS25포항씨클리프점,1.867957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상북도,포항시 남구,동해면,East,경북 포항시 남구 동해면 입암리 549-1,129.472348,36.011060
5,V7Y37-GS25포항화진점,V7Y37,GS25포항화진점,1.790957e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상북도,포항시 북구,송라면,East,경북 포항시 북구 송라면 지경리 410-7,129.373260,36.251062
6,V7Y49-GS25문경신기점,V7Y49,GS25문경신기점,1.792554e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상북도,문경시,점촌4동,East,경북 문경시 신기동 1130,128.178539,36.642537
7,V7Y34-GS25수향연암로점,V7Y34,GS25수향연암로점,1.856506e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,충청남도,천안시 서북구,성환읍,West,충남 천안시 서북구 성환읍 수향리 434-27,127.133666,36.948862
8,V7X88-GS25영통한아름점,V7X88,GS25영통한아름점,1.907360e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,경기도,수원시 영통구,매탄3동,Sudo_KW,경기 수원시 영통구 매탄동 1277,127.043638,37.256608
9,V7X89-GS25금곡센트럴점,V7X89,GS25금곡센트럴점,1.990120e+09,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경기도,수원시 권선구,금곡동,Sudo_KW,경기 수원시 권선구 금곡동 1072,126.945986,37.274688


In [44]:
origin_poc = pd.concat([origin_poc.iloc[:-n, :], tmp], axis=0)
origin_poc.to_excel(poc_list_file, index=False, encoding='utf-8-sig')

# POC Data 점검

In [45]:
print(origin_poc['X_CNTS'].isna().sum())
print(origin_poc['Area'].isna().sum())

0
0


In [46]:
## 점검 후 다시 loading ##
origin_poc = pd.read_excel(poc_list_file)

origin_poc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25859 entries, 0 to 25858
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   full_name    25859 non-null  object 
 1   poc_id       25859 non-null  object 
 2   poc_name     25859 non-null  object 
 3   Kakao_ID     24132 non-null  float64
 4   Grade        25859 non-null  object 
 5   mpocs_cd     25859 non-null  object 
 6   MNG_YN       25859 non-null  object 
 7   Branch       25859 non-null  object 
 8   Team         25859 non-null  object 
 9   SR           25859 non-null  object 
 10  New_Existed  25859 non-null  object 
 11  Area         25859 non-null  object 
 12  Sido         25859 non-null  object 
 13  GuGun        25748 non-null  object 
 14  UMD          25859 non-null  object 
 15  Region       25859 non-null  object 
 16  address      25859 non-null  object 
 17  X_CNTS       25859 non-null  float64
 18  Y_CNTS       25859 non-null  float64
dtypes: f

# Load to DB | POC info

In [47]:
server = 'onekorpmsdb001sqlkrprod.database.windows.net'
database = 'ONEKORPMSDB001db'
username='sysadmin'
password = 'Password@#SYS123'
driver= '{ODBC Driver 17 for SQL Server}'

params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + password

cnxn = pyodbc.connect(params)
cursor = cnxn.cursor()

#######################################################################
####################### 기존 데이터 삭제 ###############################
####################################################################### 
cursor.execute("TRUNCATE TABLE TBD_GSPOC_INFO ")
cnxn.commit()


db_params = urllib.parse.quote_plus(params)
# engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params), fast_executemany=True, connect_args={'connect_timeout': 10}, echo=False) 

@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    # print("FUNC call")
    if executemany:
        cursor.fast_executemany = True

origin_poc.to_sql('TBD_GSPOC_INFO', engine, index=False, if_exists="append", schema="dbo",chunksize=1000)

cnxn.commit()
cursor.close()

# Product update

In [48]:
isin_filter = data['Product'].isin(sku_list)

if isin_filter.sum() == len(data):
    origin_product = pd.read_excel(sku_list_file)
    origin_product.rename(columns = {'PKG Size' : 'PKG_SIZE','입수량' : 'UNIT_PER_PKG','In-pack' : 'IN_PACK'}, inplace = True)
    print("OK to Go")
else:
    missing_product = pd.DataFrame(missing_product).drop_duplicates()
    missing_product.columns = ['Master_CD']
    missing_product['Barcode'] = missing_product['Master_CD'].apply(lambda x: x.split('-')[0])
    missing_product['Product_Full_NM'] = missing_product['Master_CD'].apply(lambda x: x.split('-')[1])
    origin_product = pd.read_excel(sku_list_file)
    origin_product = pd.concat([origin_product, missing_product], axis=0)
    origin_product = origin_product.drop_duplicates()
    origin_product.rename(columns = {'PKG Size' : 'PKG_SIZE','입수량' : 'UNIT_PER_PKG','In-pack' : 'IN_PACK'}, inplace = True)
    origin_product.to_excel(sku_list_file, index=False, encoding='utf-8-sig')
    origin_product.tail(5)

# SKU data 점검

In [49]:
## 점검 후 다시 loading ##
origin_product = pd.read_excel(sku_list_file)

server = 'onekorpmsdb001sqlkrprod.database.windows.net'
database = 'ONEKORPMSDB001db'
username='sysadmin'
password = 'Password@#SYS123'
driver= '{ODBC Driver 17 for SQL Server}'

params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + password

cnxn = pyodbc.connect(params)
cursor = cnxn.cursor()

#######################################################################
####################### 기존 데이터 삭제 ###############################
####################################################################### 
cursor.execute("TRUNCATE TABLE TBD_GS25_SKUMAPPING ")
cnxn.commit()

db_params = urllib.parse.quote_plus(params)
# engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params), fast_executemany=True, connect_args={'connect_timeout': 10}, echo=False) 

@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    # print("FUNC call")
    if executemany:
        cursor.fast_executemany = True

origin_product.to_sql('TBD_GS25_SKUMAPPING', engine, index=False, if_exists="append", schema="dbo",chunksize=1000)

cnxn.commit()
cursor.close()

# Raw data to DB

In [50]:
data = data[['YYYYMMDD', 'Geography', 'Product', 'Unit Sales']]
data.rename(columns = {'Geography' : 'POC_CODENAME',
                       'Product' : 'SKU_CODENAME', 
                       'Unit Sales' : 'SKU_VOL'}, inplace = True)

data

,YYYYMMDD,POC_CODENAME,SKU_CODENAME,SKU_VOL
7,20211108,V1049-GS25강남동원점,8801858011017-오비)카스캔355ML,2
8,20211108,V1049-GS25강남동원점,8801858011048-오비)카스라이트캔355ML,1
9,20211108,V1049-GS25강남동원점,8801858011024-오비)카스캔500ML,4
10,20211108,V1049-GS25강남동원점,8801119740113-하이트진로)테라캔500ML,1
11,20211108,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,5
...,...,...,...,...
266111,20211114,V7V52-GS25검단본점,8801021223575-오비)덕덕구스IPA캔500ML,1
266112,20211114,V7V52-GS25검단본점,8801021220857-오비)구스아일랜드IPA캔473ML,1
266113,20211114,V7V52-GS25검단본점,8809556110933-제주)금성맥주캔500ML,1
266114,20211114,V7V52-GS25검단본점,8809556110728-문화)성산일출봉에일캔500ML,1


In [51]:
data.groupby('YYYYMMDD')['SKU_VOL'].sum()

YYYYMMDD
20211108    517344
20211109    545429
20211110    589631
20211111    613066
20211112    744381
20211113    860505
20211114    734428
Name: SKU_VOL, dtype: int64

# Sales data load to DB

In [52]:
server = 'onekorpmsdb001sqlkrprod.database.windows.net'
database = 'ONEKORPMSDB001db'
username='sysadmin'
password = 'Password@#SYS123'
driver= '{ODBC Driver 17 for SQL Server}'

params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + password

cnxn = pyodbc.connect(params)
cursor = cnxn.cursor()

cursor.execute("delete from TBF_GS_DAILYSALE where yyyymmdd >= {} and yyyymmdd <= {} ".format(data['YYYYMMDD'].min(),data['YYYYMMDD'].max()))
cnxn.commit()


db_params = urllib.parse.quote_plus(params)
# engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params), fast_executemany=True, connect_args={'connect_timeout': 10}, echo=False) 

@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    # print("FUNC call")
    if executemany:
        cursor.fast_executemany = True

data.to_sql('TBF_GS_DAILYSALE', engine, index=False, if_exists="append", schema="dbo",chunksize=1000)

cnxn.commit()
cursor.close()

# Ad-hoc (FG Extra)

In [53]:
results = data.copy()
results['Week'] = FG_week
results.groupby('YYYYMMDD')['SKU_VOL'].sum()

YYYYMMDD
20211108    517344
20211109    545429
20211110    589631
20211111    613066
20211112    744381
20211113    860505
20211114    734428
Name: SKU_VOL, dtype: int64

In [54]:
# Distribution : POC 갯수 계산
# Market share / Volume : Segment 계산
query_sku = "SELECT MASTER_CD, MLIQUOR_TYPE_CD, CORE_DETAIL_NM, NS_BRAND_NM, NS_PACKAGE_NM, SIZE, VOL_SKU FROM TBD_GS25_SKUMAPPING"

result_sku = pd.read_sql_query(query_sku, cnxn)

query_poc = "SELECT FULL_NAME, GRADE, MPOCS_CD, MNG_YN, BRANCH, TEAM, SR, NEW_EXISTED, AREA, SIDO, REGION FROM TBD_GSPOC_INFO"

result_poc = pd.read_sql_query(query_poc, cnxn)
result_poc.head()

,FULL_NAME,GRADE,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION
0,VP231-GS25성북마을점,B,2000337566,Y,특수강북팀,나르샤,M성북동,신규,LDA,서울특별시,Seoul
1,VP951-GS25장항신성점,A,2000280845,Y,의정부,카스천하,L일산동,신규,LDA,경기도,Sudo_KW
2,V2K89-GS25인계골드점,A,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,경기도,Sudo_KW
3,VT748-GS25인계원룸점,B,2000359192,Y,수원,수호천사,L탑동,기존,LDA,경기도,Sudo_KW
4,VT748-GS25인계원룸점,B,2000359192,Y,수원,수호천사,L탑동,기존,LDA,경기도,Sudo_KW


In [55]:
results_master = pd.merge(results, result_sku, left_on='SKU_CODENAME', right_on='MASTER_CD', how='left')
results_master = pd.merge(results_master, result_poc, left_on='POC_CODENAME', right_on='FULL_NAME', how='left')
results_master = results_master.query("MLIQUOR_TYPE_CD == '100'")
results_master

,YYYYMMDD,POC_CODENAME,SKU_CODENAME,SKU_VOL,Week,MASTER_CD,MLIQUOR_TYPE_CD,CORE_DETAIL_NM,NS_BRAND_NM,NS_PACKAGE_NM,...,GRADE,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION
0,20211108,V1049-GS25강남동원점,8801858011017-오비)카스캔355ML,2,W46,8801858011017-오비)카스캔355ML,100,CORE,CASS FRESH,CAN,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
1,20211108,V1049-GS25강남동원점,8801858011048-오비)카스라이트캔355ML,1,W46,8801858011048-오비)카스라이트캔355ML,100,CORE,CASS LIGHT,CAN,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
2,20211108,V1049-GS25강남동원점,8801858011024-오비)카스캔500ML,4,W46,8801858011024-오비)카스캔500ML,100,CORE,CASS FRESH,CAN,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
3,20211108,V1049-GS25강남동원점,8801119740113-하이트진로)테라캔500ML,1,W46,8801119740113-하이트진로)테라캔500ML,100,CORE,TERRA,CAN,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
4,20211108,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,5,W46,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,CAN,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053805,20211114,V7V52-GS25검단본점,8801021223575-오비)덕덕구스IPA캔500ML,1,W46,8801021223575-오비)덕덕구스IPA캔500ML,100,Premium & Above,GOOSE ISLAND,CAN,...,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,인천광역시,Sudo_KW
2053806,20211114,V7V52-GS25검단본점,8801021220857-오비)구스아일랜드IPA캔473ML,1,W46,8801021220857-오비)구스아일랜드IPA캔473ML,100,Premium & Above,GOOSE ISLAND,CAN,...,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,인천광역시,Sudo_KW
2053807,20211114,V7V52-GS25검단본점,8809556110933-제주)금성맥주캔500ML,1,W46,8809556110933-제주)금성맥주캔500ML,100,Premium & Above,JEJU,CAN,...,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,인천광역시,Sudo_KW
2053808,20211114,V7V52-GS25검단본점,8809556110728-문화)성산일출봉에일캔500ML,1,W46,8809556110728-문화)성산일출봉에일캔500ML,100,Premium & Above,JEJU,CAN,...,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,인천광역시,Sudo_KW


In [56]:
total_master = pd.DataFrame(results_master.groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique()).reset_index()
total_master.rename(columns = {'POC_CODENAME' : 'Total_POC'}, inplace=True)
total_master.head()

,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC
0,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6
1,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5
2,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6
3,W46,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1
4,W46,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2


In [57]:
total_master['Total_POC'].sum()

14701

In [58]:
value_master = results_master.query("CORE_DETAIL_NM == 'VALUE'")
value_master

,YYYYMMDD,POC_CODENAME,SKU_CODENAME,SKU_VOL,Week,MASTER_CD,MLIQUOR_TYPE_CD,CORE_DETAIL_NM,NS_BRAND_NM,NS_PACKAGE_NM,...,GRADE,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION
4,20211108,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,5,W46,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,CAN,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
16,20211108,V1049-GS25강남동원점,8801021105192-오비)필굿큐팩1.6L,2,W46,8801021105192-오비)필굿큐팩1.6L,100,VALUE,FILGOOD,PET.BOTTLE,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
123,20211108,V2A52-GS25대치삼성타운점,8801119241511-하이트)필라이트후레쉬캔500ML,3,W46,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,CAN,...,D,2000532014,Y,특수강남팀,피오나,M양재동,신규,Office,서울특별시,Seoul
132,20211108,V2A56-GS25대치은마점,8801021105154-오비)필굿캔500ML,1,W46,8801021105154-오비)필굿캔500ML,100,VALUE,FILGOOD,CAN,...,C,2000338779,Y,특수강남팀,피오나,M양재동,기존,Office,서울특별시,Seoul
133,20211108,V2A56-GS25대치은마점,8801021105567-오비)필굿SEVEN캔500ML,2,W46,8801021105567-오비)필굿SEVEN캔500ML,100,VALUE,FILGOOD,CAN,...,C,2000338779,Y,특수강남팀,피오나,M양재동,기존,Office,서울특별시,Seoul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053740,20211114,VY984-GS25홍대신안점,8801021105192-오비)필굿큐팩1.6L,4,W46,8801021105192-오비)필굿큐팩1.6L,100,VALUE,FILGOOD,PET.BOTTLE,...,B,2000093676,Y,청주,붉은악마,M복대동,기존,Residential,세종특별자치시,West
2053742,20211114,VY984-GS25홍대신안점,8801119241719-하이트)필라이트후레쉬페트1.6L,3,W46,8801119241719-하이트)필라이트후레쉬페트1.6L,100,VALUE,FILITE,PET.BOTTLE,...,B,2000093676,Y,청주,붉은악마,M복대동,기존,Residential,세종특별자치시,West
2053751,20211114,V7V52-GS25검단본점,8801021105154-오비)필굿캔500ML,6,W46,8801021105154-오비)필굿캔500ML,100,VALUE,FILGOOD,CAN,...,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,인천광역시,Sudo_KW
2053752,20211114,V7V52-GS25검단본점,8801021105567-오비)필굿SEVEN캔500ML,2,W46,8801021105567-오비)필굿SEVEN캔500ML,100,VALUE,FILGOOD,CAN,...,Not_allocated,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,인천광역시,Sudo_KW


In [59]:
# FG_extra : Distribution
FG_extra = value_master.query("SKU_CODENAME == '8801021228099-오비)필굿엑스트라캔500ML'").groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique().reset_index()
FG_extra.rename(columns = {'POC_CODENAME' : 'FG_extra_dist'}, inplace=True)


total_master = pd.merge(total_master, FG_extra, on=['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist
0,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0
1,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0
2,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,1.0
3,W46,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0
4,W46,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2892,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0
2893,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,9.0
2894,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,2.0
2895,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0


In [60]:
# FG_seven : Distribution
FG_seven = value_master.query("SKU_CODENAME == '8801021105567-오비)필굿SEVEN캔500ML'").groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique().reset_index()
FG_seven.rename(columns = {'POC_CODENAME' : 'FG_seven_dist'}, inplace=True)


total_master = pd.merge(total_master, FG_seven, on=['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist
0,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0
1,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN
2,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,1.0,1.0
3,W46,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0
4,W46,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0
2893,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,9.0,9.0
2894,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,2.0,2.0
2895,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN


In [61]:
# FG_original : Distribution
FG_original = value_master.query("(SKU_CODENAME != '8801021105567-오비)필굿SEVEN캔500ML') and (SKU_CODENAME != '8801021228099-오비)필굿엑스트라캔500ML') and (NS_BRAND_NM == 'FILGOOD')").groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique().reset_index()
FG_original.rename(columns = {'POC_CODENAME' : 'FG_original_dist'}, inplace=True)


total_master = pd.merge(total_master, FG_original, on=['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist
0,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0
1,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN,1.0
2,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,1.0,1.0,1.0
3,W46,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0
4,W46,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,1.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0
2893,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,9.0,9.0,14.0
2894,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,2.0,2.0,4.0
2895,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN,3.0


In [62]:
value_master['Volume'] = value_master['SKU_VOL'] * value_master['VOL_SKU']
value_master.head()

C:\Users\15156\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,YYYYMMDD,POC_CODENAME,SKU_CODENAME,SKU_VOL,Week,MASTER_CD,MLIQUOR_TYPE_CD,CORE_DETAIL_NM,NS_BRAND_NM,NS_PACKAGE_NM,...,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION,Volume
4,20211108,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,5,W46,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,CAN,...,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul,2500
16,20211108,V1049-GS25강남동원점,8801021105192-오비)필굿큐팩1.6L,2,W46,8801021105192-오비)필굿큐팩1.6L,100,VALUE,FILGOOD,PET.BOTTLE,...,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul,3200
123,20211108,V2A52-GS25대치삼성타운점,8801119241511-하이트)필라이트후레쉬캔500ML,3,W46,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,CAN,...,2000532014,Y,특수강남팀,피오나,M양재동,신규,Office,서울특별시,Seoul,1500
132,20211108,V2A56-GS25대치은마점,8801021105154-오비)필굿캔500ML,1,W46,8801021105154-오비)필굿캔500ML,100,VALUE,FILGOOD,CAN,...,2000338779,Y,특수강남팀,피오나,M양재동,기존,Office,서울특별시,Seoul,500
133,20211108,V2A56-GS25대치은마점,8801021105567-오비)필굿SEVEN캔500ML,2,W46,8801021105567-오비)필굿SEVEN캔500ML,100,VALUE,FILGOOD,CAN,...,2000338779,Y,특수강남팀,피오나,M양재동,기존,Office,서울특별시,Seoul,1000


In [63]:
# Volume : Value
Value_vol = value_master.query("CORE_DETAIL_NM == 'VALUE'").groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
Value_vol.rename(columns = {'Volume' : 'Value_volume'}, inplace=True)

total_master = pd.merge(total_master, Value_vol, on=['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume
0,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0,202800.0
1,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN,1.0,65800.0
2,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,1.0,1.0,1.0,90300.0
3,W46,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0,54900.0
4,W46,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,1.0,NaN,2.0,70700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0,41700.0
2893,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,9.0,9.0,14.0,174800.0
2894,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,2.0,2.0,4.0,93000.0
2895,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN,3.0,115500.0


In [64]:
# FG_extra : volume
FG_extra = value_master.query("SKU_CODENAME == '8801021228099-오비)필굿엑스트라캔500ML'").groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
FG_extra.rename(columns = {'Volume' : 'FG_extra_volume'}, inplace=True)


total_master = pd.merge(total_master, FG_extra, on=['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')

# FG_seven : Volume
FG_seven = value_master.query("SKU_CODENAME == '8801021105567-오비)필굿SEVEN캔500ML'").groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
FG_seven.rename(columns = {'Volume' : 'FG_seven_volume'}, inplace=True)


total_master = pd.merge(total_master, FG_seven, on=['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')

# FG_original : Volume
FG_original = value_master.query("(SKU_CODENAME != '8801021105567-오비)필굿SEVEN캔500ML') and (SKU_CODENAME != '8801021228099-오비)필굿엑스트라캔500ML') and (NS_BRAND_NM == 'FILGOOD')").groupby(['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
FG_original.rename(columns = {'Volume' : 'FG_original_volume'}, inplace=True)


total_master = pd.merge(total_master, FG_original, on=['Week', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master


,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
0,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0,202800.0,8500.0,500.0,59700.0
1,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN,1.0,65800.0,4000.0,NaN,11300.0
2,W46,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,1.0,1.0,1.0,90300.0,1000.0,500.0,7000.0
3,W46,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0,54900.0,9000.0,1000.0,8500.0
4,W46,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,1.0,NaN,2.0,70700.0,2500.0,NaN,19200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0,41700.0,3000.0,500.0,8800.0
2893,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,9.0,9.0,14.0,174800.0,12500.0,15500.0,74500.0
2894,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,2.0,2.0,4.0,93000.0,9000.0,10000.0,49700.0
2895,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,NaN,3.0,115500.0,4500.0,NaN,9700.0


In [65]:
total_master.fillna(0, inplace=True)

In [66]:
adhoc_original = pd.read_csv("//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/ad_hoc_fg.csv")
adhoc_original

,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
0,W43,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0,173900.0,1000.0,1000.0,50200.0
1,W43,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,0.0,1.0,57600.0,5000.0,0.0,9400.0
2,W43,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,0.0,1.0,1.0,106100.0,0.0,500.0,12500.0
3,W43,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0,50000.0,7000.0,1500.0,9000.0
4,W43,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,2.0,0.0,2.0,52300.0,4000.0,0.0,10500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17799,W45,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,1.0,0.0,3.0,43600.0,1500.0,0.0,14300.0
17800,W45,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,8.0,9.0,13.0,200600.0,13500.0,14500.0,79900.0
17801,W45,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,2.0,2.0,4.0,113800.0,9000.0,8500.0,65100.0
17802,W45,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0,0.0,3.0,115500.0,5500.0,0.0,2000.0


In [67]:
adhoc_original = pd.concat([adhoc_original, total_master], axis=0)
adhoc_original

,Week,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
0,W43,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0,173900.0,1000.0,1000.0,50200.0
1,W43,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,0.0,1.0,57600.0,5000.0,0.0,9400.0
2,W43,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,0.0,1.0,1.0,106100.0,0.0,500.0,12500.0
3,W43,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0,50000.0,7000.0,1500.0,9000.0
4,W43,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,2.0,0.0,2.0,52300.0,4000.0,0.0,10500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0,41700.0,3000.0,500.0,8800.0
2893,W46,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,9.0,9.0,14.0,174800.0,12500.0,15500.0,74500.0
2894,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,2.0,2.0,4.0,93000.0,9000.0,10000.0,49700.0
2895,W46,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,2.0,0.0,3.0,115500.0,4500.0,0.0,9700.0


In [68]:
adhoc_original.to_csv("//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/ad_hoc_fg.csv", index=False, encoding='utf-8-sig')

In [69]:
adhoc_original.groupby('Week')['Total_POC', 'FG_extra_dist', 'FG_seven_dist', 'FG_original_dist', 'Value_volume', 'FG_extra_volume', 'FG_seven_volume', 'FG_original_volume'].sum()

C:\Users\15156\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
Week,,,,,,,,
W40,14674,0.0,3409.0,11376.0,278595850.0,0.0,6907000.0,112907100.0
W41,14644,1983.0,3487.0,11441.0,287931720.0,2670500.0,7356500.0,114821300.0
W42,14654,4389.0,3379.0,11217.0,275452665.0,8926000.0,6939000.0,106743200.0
W43,14669,5042.0,3195.0,11120.0,261615900.0,10526500.0,6437000.0,100188500.0
W44,14732,5522.0,3266.0,11195.0,268869310.0,12398000.0,6784000.0,101662600.0
W45,14700,5539.0,3224.0,11095.0,299549750.0,14802000.0,7944000.0,111801200.0
W46,14701,5493.0,3123.0,11007.0,294493695.0,14919500.0,7560500.0,109525300.0


# COVID data creation

In [70]:
covid_origin = pd.read_csv('//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/COVID19/2021 4th wave/GS_region.csv')
covid_origin

,YYYYMMDD,Region,Total,ABI,HITE
0,20200101,East,62645003,28708068,14180565
1,20200101,Seoul,86968859,38579449,17210005
2,20200101,Sudo_KW,139377371,69763528,28000395
3,20200101,West,67387814,34242146,13402535
4,20200102,East,42645817,19506958,9283760
...,...,...,...,...,...
2703,20211106,West,112488511,54119379,21684725
2704,20211107,East,85891803,38994435,18414715
2705,20211107,Seoul,116774771,55385816,21505495
2706,20211107,Sudo_KW,189706127,93353929,38082940


In [71]:
sku = pd.read_excel(sku_list_file)[['Master_CD', 'MPURCHASER_NM', 'VOL_SKU']]
poc = pd.read_excel(poc_list_file)[['full_name', 'Region']]

In [72]:
df_covid = pd.merge(data, sku, left_on='SKU_CODENAME', right_on='Master_CD', how='left')
df_covid = pd.merge(df_covid, poc, left_on='POC_CODENAME', right_on='full_name', how='left')
df_covid.head()

,YYYYMMDD,POC_CODENAME,SKU_CODENAME,SKU_VOL,Master_CD,MPURCHASER_NM,VOL_SKU,full_name,Region
0,20211108,V1049-GS25강남동원점,8801858011017-오비)카스캔355ML,2,8801858011017-오비)카스캔355ML,OBC,355,V1049-GS25강남동원점,Seoul
1,20211108,V1049-GS25강남동원점,8801858011048-오비)카스라이트캔355ML,1,8801858011048-오비)카스라이트캔355ML,OBC,355,V1049-GS25강남동원점,Seoul
2,20211108,V1049-GS25강남동원점,8801858011024-오비)카스캔500ML,4,8801858011024-오비)카스캔500ML,OBC,500,V1049-GS25강남동원점,Seoul
3,20211108,V1049-GS25강남동원점,8801119740113-하이트진로)테라캔500ML,1,8801119740113-하이트진로)테라캔500ML,H&J,500,V1049-GS25강남동원점,Seoul
4,20211108,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,5,8801119241511-하이트)필라이트후레쉬캔500ML,H&J,500,V1049-GS25강남동원점,Seoul


In [73]:
df_covid = df_covid[['YYYYMMDD', 'POC_CODENAME', 'Region', 'SKU_CODENAME', 'MPURCHASER_NM', 'VOL_SKU', 'SKU_VOL']]
df_covid.head()

,YYYYMMDD,POC_CODENAME,Region,SKU_CODENAME,MPURCHASER_NM,VOL_SKU,SKU_VOL
0,20211108,V1049-GS25강남동원점,Seoul,8801858011017-오비)카스캔355ML,OBC,355,2
1,20211108,V1049-GS25강남동원점,Seoul,8801858011048-오비)카스라이트캔355ML,OBC,355,1
2,20211108,V1049-GS25강남동원점,Seoul,8801858011024-오비)카스캔500ML,OBC,500,4
3,20211108,V1049-GS25강남동원점,Seoul,8801119740113-하이트진로)테라캔500ML,H&J,500,1
4,20211108,V1049-GS25강남동원점,Seoul,8801119241511-하이트)필라이트후레쉬캔500ML,H&J,500,5


In [74]:
df_covid['VOL'] = df_covid['SKU_VOL'] * df_covid['VOL_SKU']
df_covid.head()

,YYYYMMDD,POC_CODENAME,Region,SKU_CODENAME,MPURCHASER_NM,VOL_SKU,SKU_VOL,VOL
0,20211108,V1049-GS25강남동원점,Seoul,8801858011017-오비)카스캔355ML,OBC,355,2,710
1,20211108,V1049-GS25강남동원점,Seoul,8801858011048-오비)카스라이트캔355ML,OBC,355,1,355
2,20211108,V1049-GS25강남동원점,Seoul,8801858011024-오비)카스캔500ML,OBC,500,4,2000
3,20211108,V1049-GS25강남동원점,Seoul,8801119740113-하이트진로)테라캔500ML,H&J,500,1,500
4,20211108,V1049-GS25강남동원점,Seoul,8801119241511-하이트)필라이트후레쉬캔500ML,H&J,500,5,2500


In [75]:
print(df_covid['Region'].isna().sum())
print(df_covid['MPURCHASER_NM'].isna().sum())
print(df_covid['VOL_SKU'].isna().sum())

0
0
0


In [76]:
Total = df_covid.groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
ABI = df_covid.query("MPURCHASER_NM == 'OBC'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
HITE = df_covid.query("MPURCHASER_NM == 'H&J'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()

Total = Total.reset_index()
ABI = ABI.reset_index()
HITE = HITE.reset_index()

tmp_total = Total.set_index(['YYYYMMDD', 'Region'])
tmp_abi = ABI.set_index(['YYYYMMDD', 'Region'])
tmp_hite = HITE.set_index(['YYYYMMDD', 'Region'])

tmp_total.rename(columns = {'VOL' : 'Total'}, inplace = True)
tmp_abi.rename(columns = {'VOL' : 'ABI'}, inplace = True)
tmp_hite.rename(columns = {'VOL' : 'HITE'}, inplace = True)

table = pd.merge(tmp_total, tmp_abi, left_index = True, right_index = True, how='left')
table = pd.merge(table, tmp_hite, left_index = True, right_index = True, how='left')

table = table[['Total', 'ABI', 'HITE']]
table.reset_index(inplace=True)

covid_origin = pd.concat([covid_origin, table], axis=0)
covid_origin

covid_origin.to_csv('//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/COVID19/2021 4th wave/GS_region.csv', index=False)

# COVID data | Brand

In [77]:
sku = pd.read_excel(sku_list_file)[['Master_CD', 'CORE_DETAIL_NM', 'NS_Brand_NM', 'NS_PACKAGE_NM', 'OFF_CAPACITY_NM', 'VOL_SKU']]
poc = pd.read_excel(poc_list_file)[['full_name', 'Region']]

sku_core_pet = sku.query("(CORE_DETAIL_NM == 'CORE') and (NS_PACKAGE_NM == 'PET.BOTTLE')")
sku_core_355can = sku.query("(CORE_DETAIL_NM == 'CORE') and (NS_PACKAGE_NM == 'CAN') and ((OFF_CAPACITY_NM == 355) or (OFF_CAPACITY_NM == '355') or (OFF_CAPACITY_NM == 330) or (OFF_CAPACITY_NM == 2130) or (OFF_CAPACITY_NM == 8520)  or (OFF_CAPACITY_NM == 4260))")
sku_core_500can = sku.query("(CORE_DETAIL_NM == 'CORE') and (NS_PACKAGE_NM == 'CAN') and ((OFF_CAPACITY_NM == 500) or (OFF_CAPACITY_NM == '500') or (OFF_CAPACITY_NM == 12000) or (OFF_CAPACITY_NM == 3000) or (OFF_CAPACITY_NM == '3000') or (OFF_CAPACITY_NM == 2000))")
sku_premium = sku.query("CORE_DETAIL_NM == 'Premium & Above'")

sku_core_pet.drop(['OFF_CAPACITY_NM'], axis=1, inplace=True)
sku_core_355can.drop(['OFF_CAPACITY_NM'], axis=1, inplace=True)
sku_core_500can.drop(['OFF_CAPACITY_NM'], axis=1, inplace=True)
sku_premium.drop(['OFF_CAPACITY_NM'], axis=1, inplace=True)

C:\Users\15156\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [78]:
# isin filter
isin_core_pet = data['SKU_CODENAME'].isin(sku_core_pet['Master_CD'])
result_core_pet = data[isin_core_pet]

isin_core_355can = data['SKU_CODENAME'].isin(sku_core_355can['Master_CD'])
result_core_355can = data[isin_core_355can]

isin_core_500can = data['SKU_CODENAME'].isin(sku_core_500can['Master_CD'])
result_core_500can = data[isin_core_500can]

isin_premium = data['SKU_CODENAME'].isin(sku_premium['Master_CD'])
result_premium = data[isin_premium]

In [79]:
# merge
core_pet = pd.merge(result_core_pet, sku_core_pet, left_on='SKU_CODENAME', right_on='Master_CD', how='left')
core_pet = pd.merge(core_pet, poc, left_on='POC_CODENAME', right_on='full_name', how='left')

core_355can = pd.merge(result_core_355can, sku_core_355can, left_on='SKU_CODENAME', right_on='Master_CD', how='left')
core_355can = pd.merge(core_355can, poc, left_on='POC_CODENAME', right_on='full_name', how='left')

core_500can = pd.merge(result_core_500can, sku_core_500can, left_on='SKU_CODENAME', right_on='Master_CD', how='left')
core_500can = pd.merge(core_500can, poc, left_on='POC_CODENAME', right_on='full_name', how='left')

premium = pd.merge(result_premium, sku_premium, left_on='SKU_CODENAME', right_on='Master_CD', how='left')
premium = pd.merge(premium, poc, left_on='POC_CODENAME', right_on='full_name', how='left')

In [80]:
# volume
core_pet['VOL'] = core_pet['SKU_VOL'] * core_pet['VOL_SKU']
core_355can['VOL'] = core_355can['SKU_VOL'] * core_355can['VOL_SKU']
core_500can['VOL'] = core_500can['SKU_VOL'] * core_500can['VOL_SKU']
premium['VOL'] = premium['SKU_VOL'] * premium['VOL_SKU']

In [81]:
# Core_pet
total_core_pet = core_pet.groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
cass_core_pet = core_pet.query("NS_Brand_NM == 'CASS FRESH'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
total_core_pet = total_core_pet.reset_index()
cass_core_pet = cass_core_pet.reset_index()
total_core_pet = total_core_pet.set_index(['YYYYMMDD', 'Region'])
cass_core_pet = cass_core_pet.set_index(['YYYYMMDD', 'Region'])
total_core_pet.rename(columns = {'VOL' : 'Core_pet'}, inplace=True)
cass_core_pet.rename(columns = {'VOL' : 'Cass_Fresh_pet'}, inplace=True)

table = pd.merge(total_core_pet, cass_core_pet, left_index=True, right_index=True, how='left')

# Core_355can
total_core_355can = core_355can.groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
cass_core_355can = core_355can.query("NS_Brand_NM == 'CASS FRESH'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
total_core_355can = total_core_355can.reset_index()
cass_core_355can = cass_core_355can.reset_index()
total_core_355can = total_core_355can.set_index(['YYYYMMDD', 'Region'])
cass_core_355can = cass_core_355can.set_index(['YYYYMMDD', 'Region'])
total_core_355can.rename(columns = {'VOL' : 'Core_355can'}, inplace=True)
cass_core_355can.rename(columns = {'VOL' : 'Cass_Fresh_355can'}, inplace=True)

tmp_table = pd.merge(total_core_355can, cass_core_355can, left_index=True, right_index=True, how='left')
table = pd.merge(table, tmp_table, left_index=True, right_index=True, how='left')
tmp_table = []

# Core_500can
total_core_500can = core_500can.groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
cass_core_500can = core_500can.query("NS_Brand_NM == 'CASS FRESH'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
total_core_500can = total_core_500can.reset_index()
cass_core_500can = cass_core_500can.reset_index()
total_core_500can = total_core_500can.set_index(['YYYYMMDD', 'Region'])
cass_core_500can = cass_core_500can.set_index(['YYYYMMDD', 'Region'])
total_core_500can.rename(columns = {'VOL' : 'Core_500can'}, inplace=True)
cass_core_500can.rename(columns = {'VOL' : 'Cass_Fresh_500can'}, inplace=True)

tmp_table = pd.merge(total_core_500can, cass_core_500can, left_index=True, right_index=True, how='left')
table = pd.merge(table, tmp_table, left_index=True, right_index=True, how='left')
tmp_table = []

# Premium
total_premium = premium.groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
total_premium = total_premium.reset_index()
total_premium = total_premium.set_index(['YYYYMMDD', 'Region'])
total_premium.rename(columns = {'VOL' : 'Premium'}, inplace=True)

# Premium brand
Bud = premium.query("NS_Brand_NM == 'BUDWEISER'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
Bud = Bud.reset_index()
Bud = Bud.set_index(['YYYYMMDD', 'Region'])
Bud.rename(columns = {'VOL' : 'Budweiser'}, inplace=True)

SA = premium.query("NS_Brand_NM == 'STELLA ARTOIS'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
SA = SA.reset_index()
SA = SA.set_index(['YYYYMMDD', 'Region'])
SA.rename(columns = {'VOL' : 'Stella_artois'}, inplace=True)

Hoe = premium.query("NS_Brand_NM == 'HOEGAARDEN'").groupby(['YYYYMMDD', 'Region'])['VOL'].sum()
Hoe = Hoe.reset_index()
Hoe = Hoe.set_index(['YYYYMMDD', 'Region'])
Hoe.rename(columns = {'VOL' : 'Hoegaarden'}, inplace=True)

tmp_table = pd.merge(total_premium, Bud, left_index=True, right_index=True, how='left')
tmp_table = pd.merge(tmp_table, SA, left_index=True, right_index=True, how='left')
tmp_table = pd.merge(tmp_table, Hoe, left_index=True, right_index=True, how='left')

table = pd.merge(table, tmp_table, left_index=True, right_index=True, how='left')


In [82]:
table = table.reset_index()
table

,YYYYMMDD,Region,Core_pet,Cass_Fresh_pet,Core_355can,Cass_Fresh_355can,Core_500can,Cass_Fresh_500can,Premium,Budweiser,Stella_artois,Hoegaarden
0,20211108,East,5788000,3825600,2865920,1779615,13633000,7843500,25055057,3070600,1699340,1884890
1,20211108,Seoul,7103200,4425200,2727400,1625900,13171500,7307500,35822651,4172034,3407140,1734250
2,20211108,Sudo_KW,17268600,11794200,5741330,4115515,28201500,17354500,53469007,7819330,4388250,3106960
3,20211108,West,7414800,4979400,3900715,2937625,15044000,9477500,25382968,3650842,1819890,1726110
4,20211109,East,5850000,3899200,2833430,1731335,14457000,8310000,27249367,3104824,1800260,3153100
5,20211109,Seoul,7849400,4968000,2833520,1663175,13913000,7556000,39352543,4229300,3864870,2789630
6,20211109,Sudo_KW,18016400,12387600,5950370,4217755,28973500,17356500,58024008,7997006,4918300,5044970
7,20211109,West,7561400,5066600,3764695,2780360,15041000,9457500,26969912,3610150,1912640,2552810
8,20211110,East,6493400,4220200,3083470,1938655,14845500,8491000,28959201,3420930,1940490,3434860
9,20211110,Seoul,8001000,5001200,3059315,1773935,19018500,11124000,42502309,4690580,4200540,3380470


In [83]:
origin = pd.read_excel('//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/COVID19/2021 4th wave/GS_COVID_Brand.xlsx')
origin = pd.concat([origin, table], axis=0)
origin

,YYYYMMDD,Region,Core_pet,Cass_Fresh_pet,Core_355can,Cass_Fresh_355can,Core_500can,Cass_Fresh_500can,Premium,Budweiser,Stella_artois,Hoegaarden
0,20200101,East,10393800,6492400,3258545,2260285,11008000,5694500,25038791,2270072,1646760,2556080
1,20200101,Seoul,12734200,7756400,3196065,2527955,12027000,6553500,40633628,3338864,3811020,2829740
2,20200101,Sudo_KW,28054400,19240000,6656960,5541905,21572500,13052500,54104333,5368135,4402990,4565230
3,20200101,West,11944800,8187000,4022150,3301145,11909000,7111500,25362576,2822224,1732550,2312950
4,20200102,East,5836400,3680600,2331640,1603180,7626000,4158000,18005508,1744255,1157820,1787120
...,...,...,...,...,...,...,...,...,...,...,...,...
23,20211113,West,12785400,8497800,5769295,4247930,22271500,13444500,42972375,5594840,2894830,4569330
24,20211114,East,8802000,5732600,4075075,2482515,19085500,11093500,37237874,4294420,2326970,4594780
25,20211114,Seoul,11201800,6785000,3760740,2282650,18310500,9729500,54212059,5613322,5033000,4455030
26,20211114,Sudo_KW,25420400,17043400,8126915,5819870,37535000,22378500,80210801,10595570,6694760,7488670


In [84]:
origin.to_excel("//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/COVID19/2021 4th wave/GS_COVID_Brand.xlsx", encoding='utf-sig-8', index=False)